In [183]:
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import numpy as np
import random
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
from json_tricks import dump, load
from pydub import AudioSegment, effects
import librosa
import noisereduce as nr
import tensorflow as tf
import keras
import sklearn 
import librosa
import time
import noisereduce as nr  # Import the noise reduction library






## LOADING MODELS

In [ ]:
FER2013 = load_model('Weights/FER2013_BestYet.keras')
AffectNET = load_model('Weights/AffectNet_Final.keras')
SER = load_model('Weights/best_weights.keras')

tokenizer = AutoTokenizer.from_pretrained("michellejieli/emotion_text_classifier")
textModel = AutoModelForSequenceClassification.from_pretrained("michellejieli/emotion_text_classifier")
textModel.load_state_dict(torch.load("Weights/trained_emotion_text_classifier_3.pth"))


/tmp/ipykernel_2285046/2333404743.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  textModel.load_state_dict(torch.load("Weights/trained_emotion_text_classifier_3.pth"))


<All keys matched successfully>

#### MAPPING

In [175]:
emotionMapping = {'0': 'Anger','1': 'Fear','2': 'Happiness', '3': 'Sadness', '4': 'Surprise', '5': 'Neutral'}
Textemotion_labels = ["Sadness", "Happiness", "Neutral", "Anger", "Fear", "Surprise"]
reverse_mapping = {v: k for k, v in emotionMapping.items()}

TextClasifiedLabels = {}
for i, label in enumerate(Textemotion_labels):
    if label in reverse_mapping:
        TextClasifiedLabels[i] = int(reverse_mapping[label])  # map old index to new index

print("Mapped label indices:", TextClasifiedLabels)

Mapped label indices: {0: 3, 1: 2, 2: 5, 3: 0, 4: 1, 5: 4}


In [176]:
print('SER input shape: ',SER.input_shape)
print('FER2013 input shape: ',FER2013.input_shape)
print('AffectNET input shape: ',AffectNET.input_shape)
textModel.eval()

SER input shape:  (None, 335, 50)
FER2013 input shape:  (None, 48, 48, 1)
AffectNET input shape:  (None, 96, 96, 3)


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
           

# Text Function

In [177]:
def predict_emotion(model, text):

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1)
        predicted_class = torch.argmax(probs, dim=1).item()

    predicted_emotion = Textemotion_labels[predicted_class]
    return predicted_emotion,predicted_class, probs.numpy()


## Voice Preprocess

In [193]:
def preprocess_audio_file(file_path, total_length=173056, target_sr=22050, top_db=30):
    # Load using pydub (for normalization)
    rawsound = AudioSegment.from_file(file_path)
    normalized = effects.normalize(rawsound, headroom=5.0)
    samples = np.array(normalized.get_array_of_samples(), dtype='float32')
    max_val = float(2 ** (8 * rawsound.sample_width - 1))
    samples = samples / max_val
    
    trimmed, _ = librosa.effects.trim(samples, top_db=top_db)
    
    if len(trimmed) < total_length:
        padded = np.pad(trimmed, (0, total_length - len(trimmed)), mode='constant')
    else:
        padded = trimmed[:total_length]
    
    sr_orig = rawsound.frame_rate
    if sr_orig != target_sr:
        padded = librosa.resample(padded, orig_sr=sr_orig, target_sr=target_sr)
        sr = target_sr
    else:
        sr = sr_orig
    reduced = nr.reduce_noise(y=padded, sr=sr)
    
    return reduced, sr

total_length = 173056
frame_length = 2048
hop_length = 512
expected_frames = 1 + int((total_length - frame_length) / hop_length)

def extract_features_fixed(signal, sr, frame_length=2048, hop_length=512, n_mfcc=13, expected_frames=expected_frames):
    rms = librosa.feature.rms(y=signal, frame_length=frame_length, hop_length=hop_length)
    zcr = librosa.feature.zero_crossing_rate(y=signal, frame_length=frame_length, hop_length=hop_length, center=True)
    mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length)
    mfcc_delta = librosa.feature.delta(mfcc)
    mfcc_delta2 = librosa.feature.delta(mfcc, order=2)
    
    centroid = librosa.feature.spectral_centroid(y=signal, sr=sr, hop_length=hop_length)
    contrast = librosa.feature.spectral_contrast(y=signal, sr=sr, hop_length=hop_length)
    rolloff = librosa.feature.spectral_rolloff(y=signal, sr=sr, hop_length=hop_length)
    
    features = np.vstack((zcr, rms, mfcc, mfcc_delta, mfcc_delta2, centroid, contrast, rolloff)).T
    current_frames = features.shape[0]
    
    if current_frames < expected_frames:
        pad_width = expected_frames - current_frames
        features = np.pad(features, ((0, pad_width), (0, 0)), mode='constant')
    elif current_frames > expected_frames:
        features = features[:expected_frames, :]
        
    return features
def predict_single_file(model, file_path, total_length=173056, target_sr=22050, top_db=30,
                        frame_length=2048, hop_length=512, n_mfcc=13, expected_frames=335):
    """
    Predict the emotion of a single audio file using the trained model.
    
    Parameters:
      model         : Trained model to use for prediction.
      file_path     : Path to the audio file.
      total_length  : Fixed total length for audio (in samples).
      target_sr     : Target sample rate (Hz).
      top_db        : Decibel threshold for trimming silence.
      frame_length  : Frame length for feature extraction.
      hop_length    : Hop length for feature extraction.
      n_mfcc        : Number of MFCCs to extract.
      expected_frames: Expected number of frames in the feature matrix.
    
    Returns:
      predictions   : Raw prediction probabilities (NumPy array).
      predicted_emotion: Mapped emotion label (string).
    """
    # Preprocess the audio file
    signal, sr = preprocess_audio_file(file_path, total_length=total_length, target_sr=target_sr, top_db=top_db)
    
    # Extract features with fixed number of frames
    features = extract_features_fixed(signal, sr, frame_length=frame_length, hop_length=hop_length, 
                                      n_mfcc=n_mfcc, expected_frames=expected_frames)
    # Expand dimensions to create a batch of 1 sample
    X_input = np.expand_dims(features, axis=0)
    
    # Predict using the trained model
    predictions = model.predict(X_input)
    
    # Map predicted index to emotion
    predicted_class = np.argmax(predictions, axis=1)[0]
    emotion_map = {
        0: 'neutral',
        1: 'calm',
        2: 'happy',
        3: 'sad',
        4: 'angry',
        5: 'fearful',
        6: 'disgust',
        7: 'surprised'
    }
    predicted_emotion = emotion_map.get(predicted_class, "Unknown")
    
    print("Raw prediction probabilities:", predictions)
    print("Predicted Emotion:", predicted_emotion)
    
    return predictions, predicted_emotion

# FER Function

In [186]:
def fer2013inputshape(img):
    img = cv2.resize(img, (48, 48))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0)
    return img

# AffectNet Function

In [187]:
def AffectNetinputshape(img):
    img = cv2.resize(img, (96, 96))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0)
    return img

In [188]:
#SER input shape:  (None, 335, 50)
def SERinputshape(wav):
    #wav = wav[0:335]
    wav = np.pad(wav, (0, 335-len(wav)), 'constant', constant_values=(0, 0))
    wav = np.expand_dims(wav, axis=-1)
    wav = np.expand_dims(wav, axis=0)
    return wav

    

In [189]:
img = cv2.imread("/home/group02-f24/Desktop/SadBaby.jpg")
img = img.astype('float32') / 255.0  # Normalize if needed

In [190]:
FER2013img = fer2013inputshape(img)
AffectNETimg = AffectNetinputshape(img)

### RAW PREDICTIONS

In [197]:
FER2013pred = FER2013.predict(FER2013img)
AffectNETpred = AffectNET.predict(AffectNETimg)
emotion, predclass,probs = predict_emotion(textModel,"I am so devastated!")
file_path = "/home/group02-f24/Documents/Zoghby/Audio/TESS/TESS Toronto emotional speech set data/YAF_disgust/YAF_death_disgust.wav"
predictions, predicted_emotion = predict_single_file(SER, file_path)

AffectNETpred = 1.1*AffectNETpred
FER2013pred = 0.9*FER2013pred
probs = 0.8 * probs[0, :6]

reordered_preds = np.zeros_like(probs)

for old_idx, new_idx in TextClasifiedLabels.items():
    reordered_preds[new_idx] = probs[old_idx]

probs = reordered_preds
concPred = FER2013pred + AffectNETpred + probs

print('FER2013 prediction: ',FER2013pred)
print('AffectNET prediction: ',AffectNETpred)
print('Text prediction: ',probs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Raw prediction probabilities: [[7.3396663e-08 3.6703758e-09 2.3242087e-06 2.6135036e-05 3.0111265e-05
  2.0800182e-10 9.9993312e-01 8.1801154e-06]]
Predicted Emotion: disgust
FER2013 prediction:  [[0.44925573 0.09617763 0.18659915 0.01830562 0.0162384  0.13342343]]
AffectNET prediction:  [[0.03346886 0.17315489 0.01385948 0.62941283 0.12246295 0.12764111]]
Text prediction:  [4.0775325e-04 8.8411989e-04 5.5360800e-04 7.9721123e-01 6.8269053e-04
 2.6062629e-04]


### Normalized PREDICTIONS

In [198]:
FER2013pred = np.argmax(FER2013pred, axis=1)
AffectNETpred = np.argmax(AffectNETpred, axis=1)
print('Text Classified Emotion:',emotionMapping[str(TextClasifiedLabels[predclass])])
print('FER2013 prediction: ',emotionMapping[str(FER2013pred[0])])
print('AffectNET prediction: ',emotionMapping[str(AffectNETpred[0])])
print('SER prediction: ',predicted_emotion)

Text Classified Emotion: Sadness
FER2013 prediction:  Anger
AffectNET prediction:  Sadness
SER prediction:  disgust


### Concatenated Predictions

In [171]:
finalPrediction = np.argmax(concPred, axis=1)
print(finalPrediction)
print('FER2013 prediction: ',emotionMapping[str(finalPrediction[0])])


[3]
FER2013 prediction:  Sadness
